# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645 - 675) underperformed compared to schools with smaller budgets (\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

**Note:**
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3

# loading data from python.db database
con = sqlite3.connect('python.db')

# import 'school' and 'student' tables into pandas dataframe
school_data = pd.read_sql_query("SELECT * FROM school", con)
student_data = pd.read_sql_query("SELECT * FROM student", con)


In [12]:
# join the two tables into a single dataframe
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [41]:
# Create a District Summary
district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "Overall Passing Rate": [overall_pass_rate],
    "% Passing Math": [passing_math_percentage],
    "% Passing Reading": [passing_reading_percentage]
})

In [21]:
# Total number of schools

total_schools = school_data_complete['school_name'].nunique()


In [23]:
# Total number of students
total_students = school_data_complete['student_name'].nunique()



In [25]:
# Total budget

total_budget = school_data['budget'].sum()


In [27]:
# Average math score

avg_math_score = school_data_complete['math_score'].mean()


In [29]:
# Average reading score

avg_reading_score = school_data_complete['reading_score'].mean()


In [33]:
# Overall average score
overall_pass_rate = (avg_math_score + avg_reading_score) / 2



In [43]:
# Percentage of passing math (70 or greater)

passing_math_percentage = (school_data_complete[school_data_complete['math_score'] >= 70]['student_name'].count() / total_students) * 100
passing_reading_percentage = (school_data_complete[school_data_complete['reading_score'] >= 70]['student_name'].count() / total_students) * 100
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Passing Rate,% Passing Math,% Passing Reading
0,15,32715,24649428,78.985371,81.87784,80.431606,89.775332,102.735748


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

### Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [47]:
#  Sort and display the top five schools in overall passing rate

school_summary = school_data_complete.groupby("school_name").agg(
    School_Type=('type', 'first'),
    Total_Students=('student_name', 'count'),
    Total_School_Budget=('budget', 'first'),
    Per_Student_Budget=('budget', lambda x: x.mean() / school_data_complete['student_name'].nunique()),
    Average_Math_Score=('math_score', 'mean'),
    Average_Reading_Score=('reading_score', 'mean')
)


school_summary['% Passing Math'] = school_data_complete[school_data_complete['math_score'] >= 70].groupby("school_name")['student_name'].count() / school_summary['Total_Students'] * 100
school_summary['% Passing Reading'] = school_data_complete[school_data_complete['reading_score'] >= 70].groupby("school_name")['student_name'].count() / school_summary['Total_Students'] * 100

school_summary['Overall Passing Rate'] = (school_summary['% Passing Math'] + school_summary['% Passing Reading']) / 2

school_summary.head()
top_schools = school_summary.sort_values(by='Overall Passing Rate', ascending=False).head(5)
top_schools

,School_Type,Total_Students,Total_School_Budget,Per_Student_Budget,Average_Math_Score,Average_Reading_Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,33.053829,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,31.885374,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,17.907932,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,28.045239,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,40.335442,83.274201,83.989488,93.867718,96.539641,95.203679


In [51]:
# Calculate total school budget
Total_School_Budget=('budget', 'first'),


In [55]:
# Calculate per student budget
Per_Student_Budget=('budget', lambda x: x.mean() / school_data_complete['student_name'].nunique()),


In [57]:
# Cacluate the avg math and reading score

Average_Math_Score=('math_score', 'mean'), 
Average_Reading_Score=('reading_score', 'mean')


#### Find the passing rate for math and reading (above 70 points)

In [59]:
# Find the total counts of math result
total_math_students = school_data_complete.groupby('school_name')['math_score'].count()

# Find the counts for math result in each school that pass 70 or higher
math_pass_students = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['student_name'].count()
# Calculate the math passing rate
math_passing_rate = (math_pass_students / total_math_students) * 100





In [61]:
# Find the total counts of read result

total_reading_students = school_data_complete.groupby('school_name')['reading_score'].count()

# Find the counts for read result in each school that pass 70 or higher
reading_pass_students = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['student_name'].count()


# Calculate the read passing rate

reading_passing_rate = (reading_pass_students / total_reading_students) * 100



In [63]:
# Calculate the overall passing rate (average of the math and reading passing rate)
overall_passing_rate = (math_passing_rate + reading_passing_rate) / 2
overall_passing_rate



school_name
Bailey High School       74.306672
Cabrera High School      95.586652
Figueroa High School     73.363852
Ford High School         73.804308
Griffin High School      95.265668
Hernandez High School    73.807983
Holden High School       94.379391
Huang High School        73.500171
Johnson High School      73.639992
Pena High School         95.270270
Rodriguez High School    73.293323
Shelton High School      94.860875
Thomas High School       95.290520
Wilson High School       95.203679
Wright High School       94.972222
dtype: float64

### Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [68]:
#  Sort and display the worst five schools in overall passing rate

school_performance = pd.DataFrame({
    'Math Passing Rate': math_passing_rate,
    'Reading Passing Rate': reading_passing_rate,
    'Overall Passing Rate': overall_passing_rate
})
bottom_schools = school_performance.sort_values(by='Overall Passing Rate', ascending=True).head(5)
bottom_schools


,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
school_name,,,
Rodriguez High School,66.366592,80.220055,73.293323
Figueroa High School,65.988471,80.739234,73.363852
Huang High School,65.683922,81.316421,73.500171
Johnson High School,66.057551,81.222432,73.639992
Ford High School,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [70]:
# Create table that lists the average math score for each school of each grade level.

math_scores_by_grade = school_data_complete.groupby(['school_name', 'grade'])['math_score'].mean().unstack()
math_scores_by_grade



grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [74]:
# Calculate the average math score for 9th grade in each school
math_scores_9th = student_data[student_data["grade"] == "9th"].groupby("school_name")["math_score"].mean()

print(math_scores_9th)

school_name
Bailey High School       77.083676
Cabrera High School      83.094697
Figueroa High School     76.403037
Ford High School         77.361345
Griffin High School      82.044010
Hernandez High School    77.438495
Holden High School       83.787402
Huang High School        77.027251
Johnson High School      77.187857
Pena High School         83.625455
Rodriguez High School    76.859966
Shelton High School      83.420755
Thomas High School       83.590022
Wilson High School       83.085578
Wright High School       83.264706
Name: math_score, dtype: float64


In [76]:
# Calculate the average math score for 10th grade in each school
math_scores_10th = student_data[student_data["grade"] == "10th"].groupby("school_name")["math_score"].mean()

print(math_scores_10th)


school_name
Bailey High School       76.996772
Cabrera High School      83.154506
Figueroa High School     76.539974
Ford High School         77.672316
Griffin High School      84.229064
Hernandez High School    77.337408
Holden High School       83.429825
Huang High School        75.908735
Johnson High School      76.691117
Pena High School         83.372000
Rodriguez High School    76.612500
Shelton High School      82.917411
Thomas High School       83.087886
Wilson High School       83.724422
Wright High School       84.010288
Name: math_score, dtype: float64


In [80]:
# Calculate the average math score for 11th grade in each school
math_scores_11th = student_data[student_data["grade"] == "11th"].groupby("school_name")["math_score"].mean()

print(math_scores_11th)



school_name
Bailey High School       77.515588
Cabrera High School      82.765560
Figueroa High School     76.884344
Ford High School         76.918058
Griffin High School      83.842105
Hernandez High School    77.136029
Holden High School       85.000000
Huang High School        76.446602
Johnson High School      77.491653
Pena High School         84.328125
Rodriguez High School    76.395626
Shelton High School      83.383495
Thomas High School       83.498795
Wilson High School       83.195326
Wright High School       83.836782
Name: math_score, dtype: float64


In [82]:
# Calculate the average math score for 12th grade in each school

math_scores_12th = student_data[student_data["grade"] == "12th"].groupby("school_name")["math_score"].mean()

print(math_scores_12th)


school_name
Bailey High School       76.492218
Cabrera High School      83.277487
Figueroa High School     77.151369
Ford High School         76.179963
Griffin High School      83.356164
Hernandez High School    77.186567
Holden High School       82.855422
Huang High School        77.225641
Johnson High School      76.863248
Pena High School         84.121547
Rodriguez High School    77.690748
Shelton High School      83.778976
Thomas High School       83.497041
Wilson High School       83.035794
Wright High School       83.644986
Name: math_score, dtype: float64


### Reading Score by Grade 

* Perform the same operations as above for reading scores

In [84]:
# Create table that lists the average reading score for each school of each grade level.

reading_scores_by_grade = school_data_complete.groupby(['school_name', 'grade'])['reading_score'].mean().unstack()
reading_scores_by_grade

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


In [86]:
# Calculate the average reading score for 9th grade in each school

reading_scores_9th = student_data[student_data["grade"] == "9th"].groupby("school_name")["reading_score"].mean()

print(reading_scores_9th)


school_name
Bailey High School       81.303155
Cabrera High School      83.676136
Figueroa High School     81.198598
Ford High School         80.632653
Griffin High School      83.369193
Hernandez High School    80.866860
Holden High School       83.677165
Huang High School        81.290284
Johnson High School      81.260714
Pena High School         83.807273
Rodriguez High School    80.993127
Shelton High School      84.122642
Thomas High School       83.728850
Wilson High School       83.939778
Wright High School       83.833333
Name: reading_score, dtype: float64


In [88]:
# Calculate the average reading score for 10th grade in each school

reading_scores_10th = student_data[student_data["grade"] == "10th"].groupby("school_name")["reading_score"].mean()

print(reading_scores_10th)


school_name
Bailey High School       80.907183
Cabrera High School      84.253219
Figueroa High School     81.408912
Ford High School         81.262712
Griffin High School      83.706897
Hernandez High School    80.660147
Holden High School       83.324561
Huang High School        81.512386
Johnson High School      80.773431
Pena High School         83.612000
Rodriguez High School    80.629808
Shelton High School      83.441964
Thomas High School       84.254157
Wilson High School       84.021452
Wright High School       83.812757
Name: reading_score, dtype: float64


In [90]:
# Calculate the average reading score for 11th grade in each school
reading_scores_11th = student_data[student_data["grade"] == "11th"].groupby("school_name")["reading_score"].mean()

print(reading_scores_11th)



school_name
Bailey High School       80.945643
Cabrera High School      83.788382
Figueroa High School     80.640339
Ford High School         80.403642
Griffin High School      84.288089
Hernandez High School    81.396140
Holden High School       83.815534
Huang High School        81.417476
Johnson High School      80.616027
Pena High School         84.335938
Rodriguez High School    80.864811
Shelton High School      84.373786
Thomas High School       83.585542
Wilson High School       83.764608
Wright High School       84.156322
Name: reading_score, dtype: float64


In [92]:
# Calculate the average reading score for 12th grade in each school

reading_scores_12th = student_data[student_data["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

print(reading_scores_12th)


school_name
Bailey High School       80.912451
Cabrera High School      84.287958
Figueroa High School     81.384863
Ford High School         80.662338
Griffin High School      84.013699
Hernandez High School    80.857143
Holden High School       84.698795
Huang High School        80.305983
Johnson High School      81.227564
Pena High School         84.591160
Rodriguez High School    80.376426
Shelton High School      82.781671
Thomas High School       83.831361
Wilson High School       84.317673
Wright High School       84.073171
Name: reading_score, dtype: float64


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [96]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [98]:
# Create a new column to show budget per student in each row

school_data_complete['Per_Student_Budget'] = school_data_complete['budget'] / school_data_complete['student_name'].count()


In [100]:
# Create a new column to define the spending ranges per student
school_data_complete['Spending Ranges (Per Student)'] = pd.cut(school_data_complete['Per_Student_Budget'], spending_bins, labels=group_names)




In [102]:
# Calculate the average math score within each spending range

Average_Math_Score=('math_score', 'mean')



In [104]:
# Calculate the percentage passing rate for math in each spending range

Passing_Reading=('reading_score', lambda x: (x >= 70).mean() * 100)




In [106]:
# Calculate the percentage passing rate for reading in each spending range

Passing_Math=('math_score', lambda x: (x >= 70).mean() * 100)


In [114]:
# Calculate the percentage overall passing rate in each spending range

Overall_Passing_Rate=lambda x: (((x['math_score'] >= 70).mean() + (x['reading_score'] >= 70).mean()) / 2) * 100


### Scores by School Size

* Perform the same operations as above, based on school size.

In [155]:
# Sample bins. Feel free to create your own bins.
school_size = school_data_complete.groupby('school_name')['student_name'].count().reset_index()
school_size.columns = ['school_name', 'Total Students']


school_data_complete = pd.merge(school_data_complete, school_size, on="school_name", how="left", suffixes=("", "_new"))


if 'Total Students' in school_data_complete.columns and 'Total Students_new' in school_data_complete.columns:
    school_data_complete = school_data_complete.drop(columns=['Total Students'])
    school_data_complete = school_data_complete.rename(columns={'Total Students_new': 'Total Students'})

# Define bin ranges and labels
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


school_data_complete['School Size'] = pd.cut(school_data_complete['Total Students'], bins=size_bins, labels=group_names)

# Display first few rows
print(school_data_complete[['school_name', 'Total Students', 'School Size']].head())

         school_name  Total Students        School Size
0  Huang High School            2917  Large (2000-5000)
1  Huang High School            2917  Large (2000-5000)
2  Huang High School            2917  Large (2000-5000)
3  Huang High School            2917  Large (2000-5000)
4  Huang High School            2917  Large (2000-5000)


In [157]:
# Create a new column for the bin groups
school_data_complete['School Size'] = pd.cut(school_data_complete['Total Students'], bins=size_bins, labels=group_names)

Look for the total count of test scores that pass 70% or higher




In [162]:
# math_pass_size
math_passing_count = (student_data['math_score'] >= 70).sum()

In [166]:
# read_pass_size
reading_passing_count = (student_data['reading_score'] >= 70).sum()


In [168]:
# Calculate the overall passing rate for different school size
total_passing_count = math_passing_count + reading_passing_count



### Scores by School Type

* Perform the same operations as above, based on school type.

In [ ]:
# Create bins and groups, school type {'Charter', 'District'}


Find counts of the passing 70 or higher score for the both test


In [170]:
# math pass size
math_pass_type = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['student_name'].count()




In [172]:
# reading pass size
read_pass_type = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['student_name'].count()



In [174]:
# Calculate the overall passing rate
total_students_type = school_data_complete.groupby('type')['student_name'].count()
overall_passing_rate_type = (math_pass_type + read_pass_type) / (2 * total_students_type) * 100


In [176]:
school_type_summary = pd.DataFrame({
    'Total Students': total_students_type,
    'Passing Math Count': math_pass_type,
    'Passing Reading Count': read_pass_type,
    'Overall Passing Rate (%)': overall_passing_rate_type
})

In [178]:
school_type_summary

,Total Students,Passing Math Count,Passing Reading Count,Overall Passing Rate (%)
type,,,,
Charter,12194,11426,11785,95.173856
District,26976,17944,21825,73.711818
